In [1]:
import os
import shutil
import numpy as np
import tqdm
from pycocotools.coco import COCO

In [ ]:
data_source = COCO(annotation_file='E:/LitterDetection/TACO/data/labels.json')

In [ ]:
#test labels
#label_transfer = {5: 0, 12: 1, 6: 2, 22: 3}

In [ ]:
#create all labels
gen = ((i, i-1) for i in range(1, 61))

In [ ]:
label_transfer = dict(gen)

In [ ]:
label_transfer

In [ ]:
img_ids = data_source.getImgIds()
catIds = data_source.getCatIds()
categories = data_source.loadCats(catIds)

categories.sort(key=lambda x: x['id'])

classes = {}
coco_labels = {}
coco_labels_inverse = {}
for c in categories:
    coco_labels[len(classes)] = c['id']
    coco_labels_inverse[c['id']] = len(classes)
    classes[c['name']] = len(classes)

class_num = {}

## Collect images and labels

In [ ]:
# Create a parent Directory path (were to store everything)
parent_dir = "E:/LitterDetection/TACO/"
directory1 = "tmp/images/" 
directory2 = "tmp/labels/"
path1 = os.path.join(parent_dir, directory1)
path2 = os.path.join(parent_dir, directory2)
os.makedirs(path1, exist_ok = True)
os.makedirs(path2, exist_ok = True)

In [ ]:
save_base_path  = 'tmp/labels/'
save_image_path = 'tmp/images/'

In [ ]:
data_source.loadImgs(img_ids[0])

In [ ]:
a = data_source.getAnnIds(img_ids[0])
a

In [ ]:
data_source.loadAnns(a)

In [ ]:
boxes = np.zeros((0, 5))
boxes

In [ ]:
for index, img_id in tqdm.tqdm(enumerate(img_ids), desc='change .json file to .txt file'):
    img_info = data_source.loadImgs(img_id)[0] #we need a 0 so we access the dictionary. without it, it would give a dictionary inside a list
    # Modify the path containing the folder to the file name
    save_name = img_info['file_name'].replace('/', '_')
    # Remove file extension
    file_name = save_name.split('.')[0]
    # Get the width and height of a single image
    height = img_info['height']
    width = img_info['width']
    # The storage path of the converted txt file
    save_path = save_base_path + file_name + '.txt'
    is_exist = False  
    # Record whether the picture contains the target garbage type object
    with open(save_path, mode='w') as fp:
        # Find out the number set of garbage objects according to the picture number
        annotation_id = data_source.getAnnIds(img_id)
        print('annotation:', annotation_id)
        boxes = np.zeros((0, 5))
        if len(annotation_id) == 0:  # Collection size is 0
            fp.write('')
            continue
        # Get tags from coco format
        annotations = data_source.loadAnns(annotation_id)
        lines = ''  
        
        #Record the label into yolo format after conversion
        
        # Traverse the annotations dictionary
        for annotation in annotations:
            # Get the label of the garbage object
            label = coco_labels_inverse[annotation['category_id']]
            print('label:', label)
            if label in label_transfer.keys(): #look into all the labels from the dataset 
                # If the garbage type belongs to the target garbage type, the format conversion is performed
                is_exist = True
                box = annotation['bbox']
                if box[2] < 1 or box[3] < 1:#include all images (this is just to make sure that everything is included)
                    # Skip if there is no length or width data in the original label
                    continue
                # top_x, top_y, width, height ==> cen_x, cen_y, width, height
                box[0] = round((box[0] + box[2] / 2) / width, 6) #divide by width since it is the x axis
                box[1] = round((box[1] + box[3] / 2) / height, 6)
                box[2] = round(box[2] / width, 6)
                box[3] = round(box[3] / height, 6)
                                            
                label = label_transfer[label]  # Label mapping for the current label number
                if label not in class_num.keys():
                    class_num[label] = 0
                class_num[label] = class_num.values()
                lines = lines + str(label)  # Store tags first
                for i in box:  # Restore location information
                    lines += ' ' + str(i)
                lines += '\n'  # Line up
                print('label:', label)
                
        fp.writelines(lines) #write all this new information
    if is_exist:
        # If there is an object of the target type, copy the image to the specified directory
        shutil.copy('E:/LitterDetection/TACO/data/{}'.format(img_info['file_name']), os.path.join(save_image_path, save_name))
    else:
        # If it does not exist, delete the generated label file
        os.remove(save_path)

## Split folder

In [2]:
#to split into train, validation and test
import splitfolders
splitfolders.ratio('tmp', output="taco", seed=1337, ratio=(.8, 0.1,0.1)) 

Copying files: 2972 files [00:25, 116.63 files/s]


## Train model